In [5]:
import os
from pathlib import Path

# <<< AJUSTE ESSE CAMINHO UMA VEZ >>>
PROJECT_ROOT = Path(r"C:\Users\jweydson\Desktop\PROJETO - VISAO COMPUTACIONAL").resolve()
os.chdir(PROJECT_ROOT)

print("Projeto em:", PROJECT_ROOT)


Projeto em: C:\Users\jweydson\Desktop\PROJETO - VISAO COMPUTACIONAL


In [7]:
import torch
print("CUDA disponível?", torch.cuda.is_available())


CUDA disponível? False


In [1]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import time


WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\jweydson\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Sem GPU")


True
NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
model = YOLO("yolov8n.pt")  # ou "yolov8s.pt" se sua GPU aguentar

results = model.train(
    data="visdrone.yaml",
    epochs=80,       # entre 50 e 100 (como no enunciado)
    imgsz=640,
    batch=16,
    lr0=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    device=0 if torch.cuda.is_available() else "cpu"
)


Ultralytics 8.3.228  Python-3.10.8 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=visdrone.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

In [ ]:
import os, glob, shutil

run_dir = max(glob.glob("runs/detect/train*"), key=os.path.getctime)
print("Último run:", run_dir)

os.makedirs("models", exist_ok=True)
shutil.copy(os.path.join(run_dir, "weights", "best.pt"), "models/best.pt")
print("Copiado para models/best.pt")


In [ ]:
model = YOLO("models/best.pt")

metrics = model.val(data="visdrone.yaml", split="val", imgsz=640)

print("mAP@0.5:", float(metrics.box.map50))
print("mAP@0.5:0.95:", float(metrics.box.map))
print("Precision média:", float(metrics.box.mp))
print("Recall médio:", float(metrics.box.mr))


In [ ]:
import glob, time

image_paths = glob.glob("data/visdrone/images/test/*.jpg")[:50]

start = time.time()
for img in image_paths:
    _ = model.predict(img, imgsz=640, verbose=False)
end = time.time()

fps = len(image_paths) / (end - start)
print(f"FPS médio aproximado: {fps:.2f}")


In [ ]:
from PIL import Image

run_dir = max(glob.glob("runs/detect/train*"), key=os.path.getctime)
print("Run:", run_dir)

for img_name in ["confusion_matrix.png", "PR_curve.png"]:
    img_path = os.path.join(run_dir, img_name)
    if os.path.exists(img_path):
        display(Image.open(img_path))
    else:
        print(f"{img_name} não encontrado.")


In [ ]:
test_imgs = glob.glob("data/visdrone/images/test/*.jpg")
print("Total test imgs:", len(test_imgs))

sample_img = test_imgs[0]  # pega a primeira, ou troque o índice
print("Imagem usada:", sample_img)

results = model.predict(sample_img, imgsz=640, conf=0.25)
res_plot = results[0].plot()      # BGR
res_plot_rgb = res_plot[:, :, ::-1]

plt.figure(figsize=(8, 8))
plt.imshow(res_plot_rgb)
plt.axis("off")
plt.title("Detecções YOLOv8 - VisDrone")
plt.show()
